In [261]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [262]:
train_df = pd.read_csv('./titanic/train.csv')
test_df = pd.read_csv('./titanic/test.csv')

In [263]:
combine = [train_df, test_df]

In [ ]:
train_df.head()

In [ ]:
train_df.tail()

In [ ]:
train_df.info()

In [ ]:
test_df.info()

In [ ]:
train_df.columns.values

In [ ]:
train_df.isnull().sum()

In [ ]:
test_df.isnull().sum()

In [ ]:
train_df.describe()

In [ ]:
train_df.describe(include=['O'])

In [ ]:
train_df[['Pclass', 'Survived']].groupby(['Pclass'], as_index=False).mean().sort_values(by='Survived', ascending=False)

In [ ]:
train_df[['Sex', 'Survived']].groupby(['Sex'], as_index=False).mean().sort_values(by='Survived', ascending=False)

In [ ]:
train_df[['SibSp', 'Survived']].groupby(['SibSp'], as_index=False).mean().sort_values(by='Survived', ascending=False)

In [ ]:
train_df[['Parch', 'Survived']].groupby(['Parch'], as_index=False).mean().sort_values(by='Survived', ascending=False)

In [277]:
import warnings
warnings.filterwarnings('ignore')

In [278]:
import seaborn as sns
import matplotlib

font = {'family' : 'Ubuntu', 'weight' : 'bold', 'size' : 12}
matplotlib.rc('font', **font)


In [ ]:
g = sns.FacetGrid(train_df, col='Survived')
g.map(plt.hist, 'Age', bins=20)

In [ ]:
fig = plt.figure()

train_df['Age'][train_df['Pclass'] == 1].plot(kind='kde')
train_df['Age'][train_df['Pclass'] == 2].plot(kind='kde')
train_df['Age'][train_df['Pclass'] == 3].plot(kind='kde')
plt.xlabel(u'Age')
plt.ylabel(u'密度')
plt.title(u'各等级的乘客年龄分布')
plt.legend([u'1等舱', u'2等舱', u'3等舱'], loc='best')
plt.show()

In [ ]:
fig = plt.figure()
survived_0 = train_df['Pclass'][train_df['Survived'] == 0].value_counts()
survived_1 = train_df['Pclass'][train_df['Survived'] == 1].value_counts()
df = pd.DataFrame({u'surviced': survived_1, u'unsurvived': survived_0})
df.plot(kind='bar', stacked=True)
plt.title(u'Degree - Survived')
plt.xlabel(u'Degree')
plt.ylabel(u'count')
plt.show()

In [ ]:
fig = plt.figure()
fig.set(alpha=0.2)

survived_m = train_df.Survived[train_df.Sex == 'male'].value_counts()
survived_f = train_df.Survived[train_df.Sex == 'female'].value_counts()

df = pd.DataFrame({'male': survived_m, 'female': survived_f})
df.plot(kind='bar', stacked=True)
plt.show()


In [ ]:
fig = plt.figure()

survived_0 = train_df['Embarked'][train_df['Survived'] == 0].value_counts()
survived_1 = train_df['Embarked'][train_df['Survived'] == 1].value_counts()
df = pd.DataFrame({u'survived': survived_1, u'unsurvived': survived_0})
df.plot(kind='bar', stacked=True)
plt.title(u'Embarked - Survived')
plt.xlabel(u'Embarked')
plt.ylabel(u'count')
plt.show()

In [289]:
train_df = train_df.drop(['Ticket', 'Cabin'], axis=1)
test_df = test_df.drop(['Ticket', 'Cabin'], axis=1)
combine = [train_df, test_df]

In [ ]:
train_df['Name'].head()

In [ ]:
for dataset in combine:
    dataset['Title'] = dataset.Name.str.extract('([A-Za-z]+)\.', expand=False)

pd.crosstab(train_df['Title'], train_df['Sex']).sort_values(by='female', ascending=False)

In [ ]:
for dataset in combine:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess', 'Capt', 'Col', 'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')
train_df[['Title', 'Survived']].groupby(['Title'], as_index=False).mean()

In [ ]:
title_mapping = {'Mr': 1, 'Miss': 2, 'Mrs': 3, 'Master': 4, 'Rare': 5}
for dataset in combine:
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(0)
train_df.head()

In [295]:
train_df = train_df.drop(['Name', 'PassengerId'], axis=1)
test_df = test_df.drop(['Name'], axis=1)
combine = [train_df, test_df]

In [ ]:
for dataset in combine:
    dataset['Sex'] = dataset['Sex'].map({'female':1, 'male':0}).astype(int)
train_df.head()

In [ ]:
grid = sns.FacetGrid(train_df, col='Pclass',hue='Sex')
grid.map(plt.hist, 'Age', alpha=.5, bins=20)
grid.add_legend()



In [ ]:
# 创建空数组
guess_ages = np.zeros((2,3))
guess_ages

# 遍历 Sex (0 或 1) 和 Pclass (1, 2, 3) 来计算六种组合的 Age 猜测值
for dataset in combine:
    # 第一个for循环计算每一个分组的Age预测值
    for i in range(0, 2):
        for j in range(0, 3):
            guess_df = dataset[(dataset['Sex'] == i) & \
                                  (dataset['Pclass'] == j+1)]['Age'].dropna()
            age_guess = guess_df.median()

            # 将随机年龄浮点数转换为最接近的 0.5 年龄（四舍五入）
            guess_ages[i,j] = int( age_guess/0.5 + 0.5 ) * 0.5
            
    # 第二个for循环对空值进行赋值        
    for i in range(0, 2):
        for j in range(0, 3):
            dataset.loc[ (dataset.Age.isnull()) & (dataset.Sex == i) & (dataset.Pclass == j+1),\
                    'Age'] = guess_ages[i,j]

    dataset['Age'] = dataset['Age'].astype(int)
train_df.head()

In [ ]:
# 创建年龄段,并确定其与Survived的相关性
# 一般在建立分类模型时，需要对连续变量离散化，特征离散化后，模型会更稳定，降低了模型过拟合的风险
train_df['AgeBand'] = pd.cut(train_df['Age'], 5)  # 将年龄分割为5段,等距分箱
train_df[['AgeBand', 'Survived']].groupby(['AgeBand'], as_index=False).mean().sort_values(by='AgeBand', ascending=True)

In [ ]:
# 将这些年龄区间替换为序数
for dataset in combine:    
    dataset.loc[ dataset['Age'] <= 16, 'Age'] = 0
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
    dataset.loc[ dataset['Age'] > 64, 'Age'] = 4
train_df.head()

In [ ]:
# 删除训练集中的AgeBand特征
train_df = train_df.drop(['AgeBand'], axis=1)  
combine = [train_df, test_df]
train_df.head()

In [ ]:
for dataset in combine:
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1

train_df[['FamilySize', 'Survived']].groupby(['FamilySize'], as_index=False).mean().sort_values(by='Survived', ascending=False)

In [ ]:
# 创建新特征IsAlone
for dataset in combine:
    dataset['IsAlone'] = 0
    dataset.loc[dataset['FamilySize'] == 1, 'IsAlone'] = 1

train_df[['IsAlone', 'Survived']].groupby(['IsAlone'], as_index=False).mean()

In [ ]:
train_df = train_df.drop(['Parch', 'SibSp', 'FamilySize'], axis=1)
test_df = test_df.drop(['Parch', 'SibSp', 'FamilySize'], axis=1)
combine = [train_df, test_df]

train_df.head()

In [ ]:
freq_port = train_df.Embarked.dropna().mode()[0] 
for dataset in combine:
    dataset['Embarked'] = dataset['Embarked'].fillna(freq_port)
    
train_df[['Embarked', 'Survived']].groupby(['Embarked'], as_index=False).mean().sort_values(by='Survived', ascending=False)

In [ ]:
for dataset in combine:
    dataset['Embarked'] = dataset['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)

train_df.head()

In [ ]:
# 测试集中Fare有一个缺失值，用中位数进行填补
test_df['Fare'].fillna(test_df['Fare'].dropna().median(), inplace=True)
test_df.head()

In [ ]:
plt.hist(train_df['Fare'])

In [ ]:
train_df['FareBand'] = pd.qcut(train_df['Fare'], 4)  # 根据样本分位数进行分箱，等频分箱
train_df[['FareBand', 'Survived']].groupby(['FareBand'], as_index=False).mean().sort_values(by='FareBand', ascending=True)

In [ ]:
for dataset in combine:
    dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] = 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare']   = 2
    dataset.loc[ dataset['Fare'] > 31, 'Fare'] = 3
    dataset['Fare'] = dataset['Fare'].astype(int)

train_df = train_df.drop(['FareBand'], axis=1)
combine = [train_df, test_df]
    
train_df.head()

In [ ]:
test_df.head()

In [ ]:
X_train = train_df.drop("Survived", axis=1)
Y_train = train_df["Survived"]
X_test  = test_df.drop("PassengerId", axis=1)
X_train.shape, Y_train.shape, X_test.shape

In [ ]:
train_df.drop("Survived", axis=1)
train_df.head()

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train, Y_train)
y_predict = knn.predict(X_test)
score_knn = knn.score(X_train, Y_train)
score_knn

In [ ]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression()
log_reg.fit(X_train, Y_train)
y_predict = log_reg.predict(X_test)  
score_log = log_reg.score(X_train, Y_train)
score_log

In [ ]:
from sklearn.svm import SVC

svc = SVC()
svc.fit(X_train, Y_train)
y_predict = svc.predict(X_test)
score_svc = svc.score(X_train, Y_train)
score_svc

In [ ]:
from sklearn.naive_bayes import GaussianNB

gaussian = GaussianNB()
gaussian.fit(X_train, Y_train)
y_predict = gaussian.predict(X_test)
score_gaussian = gaussian.score(X_train, Y_train)
score_gaussian

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dt_clf = DecisionTreeClassifier()
dt_clf.fit(X_train, Y_train)
y_predict = dt_clf.predict(X_test)
score_dt_clf = dt_clf.score(X_train, Y_train)
score_dt_clf

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier(n_estimators=100)
rf_clf.fit(X_train, Y_train)
y_predict = rf_clf.predict(X_test)
score_rf_clf = rf_clf.score(X_train, Y_train)
score_rf_clf

In [ ]:
models = pd.DataFrame({
    'Model': [ 'KNN', 'Logistic Regression', 'SVM',
              'Naive Bayes', 'Decision Tree','Random Forest' ],
    'Score': [score_knn, score_log, score_svc, 
              score_gaussian, score_dt_clf, score_rf_clf]})
models.sort_values(by='Score', ascending=False)

In [321]:
submission = pd.DataFrame({
        "PassengerId": test_df["PassengerId"],
        "Survived": y_predict
    })
submission.to_csv('./submission.csv', index=False)